In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import pandas as pd
from ast import literal_eval
from pathlib import Path

In [49]:
from ecephys.scoring import visbrain_hypnogram_to_datetime, write_datetime_hypnogram
from ecephys_analyses.data import paths
from ecephys.sglx_utils import load_timeseries

In [50]:
def load_sr_chans(sr_chans_path):
    df = pd.read_csv(sr_chans_path)
    df.sr_chans = df.sr_chans.apply(lambda x: [] if pd.isnull(x) else list(literal_eval(x)))
    df["duration"] = df.apply(lambda row: row.end_time - row.start_time, axis=1)
    
    return df

In [55]:
def write_datetime_format(sr_chans_dt, path):
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    sr_chans_dt.to_csv(
        path,
        columns=["sr_chans", "start_time", "end_time", "duration"],
        sep="\t",
        index=False
    )

In [56]:
def write_experiment(subject, experiment):
    bin_paths = paths.get_sglx_style_datapaths(subject, experiment, "all", ext="lf.bin")
    sr_chans_paths = paths.get_sglx_style_datapaths(subject, experiment, "all", ext="sr_chans.csv")
    sr_chans_dt_paths = paths.get_sglx_style_datapaths(subject, experiment, "all", ext="sr_chans.tsv")
    
    all_sr_chans = [load_sr_chans(path) for path in sr_chans_paths]
    sigs = [load_timeseries(path, chans=[0], start_time=0, end_time=1) for path in bin_paths]
        
    for sig, sr_chans, sr_chans_dt_path in zip(sigs, all_sr_chans, sr_chans_dt_paths):
        file_start_dt = pd.to_datetime(sig.fileCreateTime)
        sr_chans_dt = visbrain_hypnogram_to_datetime(sr_chans, file_start_dt)
        write_datetime_format(sr_chans_dt, sr_chans_dt_path)

In [57]:
write_experiment(subject="Allan", experiment="sleep-homeostasis")

nChan: 385, nFileSamp: 4242731
nChan: 385, nFileSamp: 361262
nChan: 385, nFileSamp: 3863847
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 4274822
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 6626338
nChan: 385, nFileSamp: 8999999
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 1747894
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8998952
nChan: 385, nFileSamp: 8999999
nChan: 385, nFileSamp: 6172746
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 3449805
nChan: 385, nFileSamp: 4577439
nChan: 385, nFileSamp: 3682820
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 4821759
nChan: 385, nFileSamp: 7183437
nChan: 385, nFileSamp: 8999999
nChan: 385, nFileSamp: 8999998
nChan: 38